<a href="https://colab.research.google.com/github/curtiscu/LYIT/blob/master/TestingStatsNFeaturesCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing Refactor GatheringBaseMetrics

Used to test code built up/ worked out in the following notebook...
* https://github.com/curtiscu/LYIT/blob/master/GatheringBaseMetrics_1.ipynb

# Setup env


In [1]:
# print all cell output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


## Google drive access

In [2]:
# mount google drive

from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
# test, peek at data
! ls -al '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/'

# test, modules from local  'E:\Google Drive\LYIT\Dissertation\modules'
! ls -al '/content/drive/My Drive/LYIT/Dissertation/modules/'

total 35
-rw------- 1 root root 2589 Apr 27 12:01 10_soul-groove10_102_beat_4-4.mid
-rw------- 1 root root 4793 Apr 27 12:01 1_funk-groove1_138_beat_4-4.mid
-rw------- 1 root root 3243 Apr 27 12:01 2_funk-groove2_105_beat_4-4.mid
-rw------- 1 root root 4466 Apr 27 12:01 3_soul-groove3_86_beat_4-4.mid
-rw------- 1 root root 2551 Apr 27 12:01 4_soul-groove4_80_beat_4-4.mid
-rw------- 1 root root 3798 Apr 27 12:01 5_funk-groove5_84_beat_4-4.mid
-rw------- 1 root root 3760 Apr 27 12:01 6_hiphop-groove6_87_beat_4-4.mid
-rw------- 1 root root 1894 Apr 27 12:01 7_pop-groove7_138_beat_4-4.mid
-rw------- 1 root root 2437 Apr 27 12:01 8_rock-groove8_65_beat_4-4.mid
-rw------- 1 root root 3448 Apr 27 12:01 9_soul-groove9_105_beat_4-4.mid
total 36
-rw------- 1 root root 21765 Jun 26 11:24 data_prep.py
drwx------ 2 root root  4096 May 10 13:31 __pycache__
-rw------- 1 root root  1931 Jun 26 11:27 stats_n_features.py
-rw------- 1 root root  8416 Jun  2 21:24 visualizations.py


## Auto reload module

Now using library code I've created and saved to google drive which is automatically pushed to the cloud and made available to the colab env. The autoreload stuff below should help imports to 'reimport' to load changes to the library code.

It's not the quickest/ most reliable, so if in a hurry, brute force loading of changes by restarting the runtime.

In [4]:
# tool to auto reload modules.
%load_ext autoreload

# config to auto-reload all modules, handy to make 
# writing and testing modules much easier.
%autoreload 2

## Imports and accessing lib functions

In [5]:
# install required libs
!pip install mido

     |████████████████████████████████| 61kB 1.9MB/s 


In [6]:
# import my modules
import sys
sys.path.append('/content/drive/My Drive/LYIT/Dissertation/modules/')
import data_prep as dp

>> LOADING custom module, when: 2020-06-26 11:28:58.021182, module name: data_prep


In [7]:
# imports
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np


# object that provides colours for charts
from itertools import cycle



In [8]:
# testing auto reload of modules 
dp.test_function_call('bling')

Test function in data_prep.py called and worked! when: 2020-06-26 11:28:58.070556,  param:bling


In [14]:
import stats_n_features as sf

In [15]:
sf.test_function_call('hello')

Test function in stats_n_features called and worked! when: 2020-06-26 11:32:28.561504,  param:hello


## Pandas display options

In [9]:
def set_pandas_display_options() -> None:
    # Ref: https://stackoverflow.com/a/52432757/
    display = pd.options.display

    display.max_columns = 1000
    display.max_rows = 2000
    display.max_colwidth = 1000
    display.width = None
    # display.precision = 2  # set as needed

set_pandas_display_options()
#pd.reset_option('all')


# Testing stats_n_features.py

## Load some data

In [16]:
file_1 = '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/1_funk-groove1_138_beat_4-4.mid'
file_7 = '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer7/eval_session/1_funk-groove1_138_beat_4-4.mid'

file_df, f, mtt = dp.load_file(file_1)  # load data

>> LOADING custom module, when: 2020-06-26 12:46:06.860626, module name: stats_n_features
FILE name: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/1_funk-groove1_138_beat_4-4.mid
    loaded file: <midi file '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/1_funk-groove1_138_beat_4-4.mid' type 0, 1 tracks, 1300 messages>
    time sig: <meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
    tempo: <meta message set_tempo tempo=434783 time=0>
    track count: 1, tracks: [<midi track 'MIDI' 1300 messages>]
    MIDI file type: 0
    > processing track: <midi track 'MIDI' 1300 messages>
    note_on span - first tick: 3 , last tick: 30658 
    good instruments: 6, {36.0: 'Bass Drum 1 (36)', 38.0: 'Acoustic Snare (38)', 42.0: 'Closed Hi Hat (42)', 43.0: 'High Floor Tom (43)', 49.0: 'Crash Cymbal 1 (49)', 51.0: 'Ride Cymbal 1 (51)'}
    bar info - bars in file: 16, bar quantize

## Test frequency table code

In [35]:
freq_table = sf.frequency_table(file_df)

>> LOADING custom module, when: 2020-06-26 12:51:37.352022, module name: stats_n_features


bar_beat_number,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
note,,,,,,,,,,,,,,,,
Bass Drum 1 (36),12,0,16,4,0,0,0,0,0,0,15,6,0,1,0,0
Acoustic Snare (38),10,5,0,0,16,0,0,16,0,17,0,2,8,0,8,19
Closed Hi Hat (42),15,2,11,0,16,2,14,0,17,3,13,0,16,1,13,1
High Floor Tom (43),0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
Crash Cymbal 1 (49),1,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0
Ride Cymbal 1 (51),16,0,16,3,16,0,16,0,17,0,12,0,16,0,16,0


In [27]:
display(freq_table)

bar_beat_number,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
note,,,,,,,,,,,,,,,,
Bass Drum 1 (36),12,0,16,4,0,0,0,0,0,0,15,6,0,1,0,0
Acoustic Snare (38),10,5,0,0,16,0,0,16,0,17,0,2,8,0,8,19
Closed Hi Hat (42),15,2,11,0,16,2,14,0,17,3,13,0,16,1,13,1
High Floor Tom (43),0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
Crash Cymbal 1 (49),1,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0
Ride Cymbal 1 (51),16,0,16,3,16,0,16,0,17,0,12,0,16,0,16,0
